# Pointers
One of the biggest hurdles when transitioning from Python to Cython is the use of pointers in C code, which is necessary to write clean Cython programs.


In C, every variable has a memory location and every memory location has a memory address that can be accessed using the `&` symbol.

In [2]:
%load_ext Cython

In [3]:
%%cython
# from libc.stdio cimport printf

# this works, but unfortunately prints to the command line and not the notebook
# https://github.com/ipython/ipython/issues/1230
# cdef int variable_1 = 10
# cdef int variable_2 = 20
# printf("Address of variable_1: %x\n", &variable_1);
# printf("Address of variable_2: %x\n", &variable_2);

# this throws an error, is there any way I can fix this?
cdef int variable_1 = 10
cdef int variable_2 = 20
print &variable_1
print &variable_2


Error compiling Cython file:
------------------------------------------------------------
...
# printf("Address of variable_2: %x\n", &variable_2);

# this throws an error, is there any way I can fix this?
cdef int variable_1 = 10
cdef int variable_2 = 20
print &variable_1
     ^
------------------------------------------------------------

/Users/nelsonliu/.ipython/cython/_cython_magic_e74bf1eb9fd67c9f5ac2f4184431bed8.pyx:13:6: Cannot convert 'int *' to Python object

Error compiling Cython file:
------------------------------------------------------------
...

# this throws an error, is there any way I can fix this?
cdef int variable_1 = 10
cdef int variable_2 = 20
print &variable_1
print &variable_2
     ^
------------------------------------------------------------

/Users/nelsonliu/.ipython/cython/_cython_magic_e74bf1eb9fd67c9f5ac2f4184431bed8.pyx:14:6: Cannot convert 'int *' to Python object
